In [13]:
seed_value = 42
import os

import sox

os.environ['PYTHONHASHSEED'] = str(seed_value)
import random

random.seed(seed_value)
import numpy as np

np.random.seed(seed_value)
import tensorflow as tf

tf.random.set_seed(seed_value)

import opensmile

path = 'data'  # download and extract files from https://github.com/aliyzd95/ShEMO-Modification/raw/main/shemo.zip
emo_codes = {"A": 0, "W": 1, "H": 2, "S": 3, "N": 4, "F": 5}
emo_labels = ["anger", "surprise", "happiness", "sadness", "neutral", "fear"]


def get_emotion_label(file_name):
    emo_code = file_name[3]
    return emo_codes[emo_code]


def opensmile_Functionals():
    feature_extractor = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals,
        verbose=True, num_workers=None,
        sampling_rate=16000, resample=True,
    )
    features = []
    emotions = []
    for file in os.listdir(path):
        if emo_labels[get_emotion_label(file)] != 'fear':
            df = feature_extractor.process_file(f'{path}/{file}')
            features.append(df)
            emotions.append(get_emotion_label(file))
    features = np.array(features).squeeze()
    emotions = np.array(emotions)
    return features, emotions

In [14]:
seed_value = 42
import os

os.environ['PYTHONHASHSEED'] = str(seed_value)
import random

random.seed(seed_value)
import numpy as np

np.random.seed(seed_value)
import tensorflow as tf

tf.random.set_seed(seed_value)

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsOneClassifier
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import make_pipeline
import warnings
#import opensmile_preprocessing
#%run opensmile_preprocessing.ipynb
#from opensmile_preprocessing import opensmile_Functionals, emo_labels
#from ipynb.fs.full.<opensmile_preprocessing> import <opensmile_Functionals>
#from ipynb.fs.full.<opensmile_preprocessing> import <emo_labels>

warnings.filterwarnings("ignore")


def generate_confusion_matrix(cnf_matrix, classes, normalize=False, title='Confusion matrix'):
    if normalize:
        cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(cnf_matrix, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cnf_matrix.max() / 2.
    for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
        plt.text(j, i, format(cnf_matrix[i, j], fmt), horizontalalignment="center",
                 color="white" if cnf_matrix[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return cnf_matrix


def plot_confusion_matrix(predicted_labels_list, y_test_list):
    cnf_matrix = confusion_matrix(y_test_list, predicted_labels_list)
    np.set_printoptions(precision=2)
    plt.figure()
    generate_confusion_matrix(cnf_matrix, classes=emo_labels, normalize=True, title='SVM + eGeMAPS')
    plt.show()


def svm(X, y):
    cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed_value)
    cv_inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed_value)
    model = SVC()
    ovo = OneVsOneClassifier(model)
    space = dict()
    space['estimator__C'] = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
    space['estimator__gamma'] = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
    search = BayesSearchCV(ovo, space, scoring='recall_macro', cv=cv_inner, n_jobs= None, verbose=0)
    pipeline = make_pipeline(StandardScaler(), search)
    scores = cross_validate(pipeline, X, y, scoring=['recall_macro', 'accuracy'], cv=cv_outer, n_jobs=None, verbose=2)
    print('____________________ Support Vector Machine ____________________')
    print(f"Weighted Accuracy: {np.mean(scores['test_accuracy'] * 100)}")
    print(f"Unweighted Accuracy: {np.mean(scores['test_recall_macro']) * 100}")


X, y = opensmile_Functionals()


N_SAMPLES = X.shape[0]

perm = np.random.permutation(N_SAMPLES)
X = X[perm]
y = y[perm]

if __name__ == '__main__':
    svm(X, y)

    # source:
    # Accuracy: 72.95645139237044
    # UAR: 58.66650383394545
    
    # male:
    # Accuracy: 76.35305021907651
    # UAR: 55.13157619478546
    # Time: 3.6 + 16.7 
   
    # female:
    # Accuracy: 69.86364814095091
    # UAR: 60.62245031009332
    # Time: 2.5 + 12.8
    
    # modified shemo data:
    # Accuracy: 74.40915409507315
    # UAR: 59.16878570719483
    # Time: 9.7 + 47.5
    
    # shemo data:
    # Accuracy: 74.40915409507315
    # UAR: 59.16878570719483
    # Time: 8.4 + 44.0
    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total= 8.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.4min remaining:    0.0s


[CV] ................................................. , total= 9.0min
[CV]  ................................................................
[CV] ................................................. , total= 9.2min
[CV]  ................................................................
[CV] ................................................. , total= 8.6min
[CV]  ................................................................
[CV] ................................................. , total= 8.9min
____________________ Support Vector Machine ____________________
Weighted Accuracy: 74.40915409507315
Unweighted Accuracy: 59.16878570719483


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 44.0min finished
